# Generate Data of the CNN
Notebook used to generate the data from both trained CNN generators

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/c147_project/')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import time
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
# get the device type of machine
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CNNGenerator(nn.Module):
  def __init__(self,input_dim,hidden_dims):
    super(CNNGenerator,self).__init__()
    self.input_dim = input_dim
    self.l1 = nn.Sequential(
        nn.ConvTranspose1d(self.input_dim,hidden_dims[0],4,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[0]),
        nn.ReLU(True)
    )
    self.l2 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[0],hidden_dims[1],4,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[1]),
        nn.ReLU(True)
    )
    self.l3 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[1],hidden_dims[2],4,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[2]),
        nn.ReLU(True)
    )
    self.l4 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[2],hidden_dims[3],7,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[3]),
        nn.ReLU(True)
    )
    self.l5 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[3],22,4,2,0, bias = False),
        nn.Tanh()
    )
    
  def forward(self,input):
    l1 = self.l1(input) 
    l2 = self.l2(l1)
    l3 = self.l3(l2)
    l4 = self.l4(l3)
    l5 = self.l5(l4)
    # print(l1.shape[2],l2.shape[2],l3.shape[2],l4.shape[2],l5.shape[2])
    return l5

class CNNLSTMGenerator(nn.Module):
  def __init__(self,input_dim,hidden_dims):
    super(CNNLSTMGenerator,self).__init__()
    self.input_dim = input_dim
    self.l1 = nn.Sequential(
        nn.ConvTranspose1d(self.input_dim,hidden_dims[0],4,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[0]),
        nn.ReLU(True)
    )
    self.l2 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[0],hidden_dims[1],4,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[1]),
        nn.ReLU(True)
    )
    self.l3 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[1],hidden_dims[2],4,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[2]),
        nn.ReLU(True)
    )
    self.l4 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[2],hidden_dims[3],7,2,0, bias = False),
        nn.BatchNorm1d(hidden_dims[3]),
        nn.ReLU(True)
    )
    self.l5 = nn.Sequential(
        nn.ConvTranspose1d(hidden_dims[3],10,4,2,0, bias = False),
        nn.ReLU(True)
    )
    self.l6 = nn.LSTM(10,22,2, batch_first = True) 


  def forward(self,input):
    l1 = self.l1(input) 
    l2 = self.l2(l1)
    l3 = self.l3(l2)
    l4 = self.l4(l3)
    l5 = self.l5(l4)
    l6,_ = self.l6(l5.view(l5.shape[0],l5.shape[2],l5.shape[1]))
    l6 = l6.reshape(l6.shape[0],l6.shape[2],l6.shape[1])
    return l6


In [ ]:
num_trials = 1000


In [ ]:
path = 'Models/CNN+LSTM-CNN/Generator_5000.pth'
sd = torch.load(path, map_location=device)
model = CNNLSTMGenerator(100,[220,154,88,44])
model.load_state_dict(sd)
model.eval()

CNNLSTMGenerator(
  (l1): Sequential(
    (0): ConvTranspose1d(100, 220, kernel_size=(4,), stride=(2,), bias=False)
    (1): BatchNorm1d(220, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l2): Sequential(
    (0): ConvTranspose1d(220, 154, kernel_size=(4,), stride=(2,), bias=False)
    (1): BatchNorm1d(154, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l3): Sequential(
    (0): ConvTranspose1d(154, 88, kernel_size=(4,), stride=(2,), bias=False)
    (1): BatchNorm1d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l4): Sequential(
    (0): ConvTranspose1d(88, 44, kernel_size=(7,), stride=(2,), bias=False)
    (1): BatchNorm1d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l5): Sequential(
    (0): ConvTranspose1d(44, 10, kernel_size=(4,), stride=(2,), bias=False)
    (1): Re

In [ ]:
labels_store = torch.randint(0, 4, (num_trials, 1))

In [ ]:
noise = torch.rand((num_trials, model.input_dim - 1))
noise = torch.cat([noise, labels_store], dim=1)
noise = noise.view(-1, model.input_dim, 1)

In [ ]:
fake_data = model(noise)
fake_data.shape

torch.Size([1000, 22, 100])

In [ ]:
# saving data
label_to_file = labels_store.numpy()
data_to_file = fake_data.detach().numpy()

#
np.save('Models/Fake_Data/CNN+LSTM-CNN_data.npy', data_to_file)
np.save('Models/Fake_Data/CNN+LSTM-CNN_labels.npy', label_to_file)

In [ ]:
path = 'Models/CNN-CNN/Generator_5000.pth'
sd = torch.load(path, map_location=device)
model = CNNGenerator(100,[220,154,88,44])
model.load_state_dict(sd)
model.eval()

CNNGenerator(
  (l1): Sequential(
    (0): ConvTranspose1d(100, 220, kernel_size=(4,), stride=(2,), bias=False)
    (1): BatchNorm1d(220, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l2): Sequential(
    (0): ConvTranspose1d(220, 154, kernel_size=(4,), stride=(2,), bias=False)
    (1): BatchNorm1d(154, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l3): Sequential(
    (0): ConvTranspose1d(154, 88, kernel_size=(4,), stride=(2,), bias=False)
    (1): BatchNorm1d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l4): Sequential(
    (0): ConvTranspose1d(88, 44, kernel_size=(7,), stride=(2,), bias=False)
    (1): BatchNorm1d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (l5): Sequential(
    (0): ConvTranspose1d(44, 22, kernel_size=(4,), stride=(2,), bias=False)
    (1): Tanh()

In [ ]:
labels_store = torch.randint(0, 4, (num_trials, 1))

In [ ]:
noise = torch.rand((num_trials, model.input_dim - 1))
noise = torch.cat([noise, labels_store], dim=1)
noise = noise.view(-1, model.input_dim, 1)

In [ ]:
fake_data = model(noise)
fake_data.shape

torch.Size([1000, 22, 100])

In [ ]:
# saving data
label_to_file = labels_store.numpy()
data_to_file = fake_data.detach().numpy()

#
np.save('Models/Fake_Data/CNN-CNN_data.npy', data_to_file)
np.save('Models/Fake_Data/CNN-CNN_labels.npy', label_to_file)